In [1]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"


In [10]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "azerty92")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity() 

In [23]:
def create_person(tx, name):  
    result = tx.run(
        "MERGE (:Person {name: $name, school:'Telecom Paris'})",  
        name=name  
    )
    summary = result.consume()
    return summary

In [25]:
with driver.session(database="foo") as session:
    summary = session.execute_write(create_person, name="Florian")  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 1 nodes in 4 ms.


In [ ]:
def import_country(tx):  
    result = tx.run(
        "LOAD CSV WITH HEADERS FROM 'file:///fips.csv' AS row WITH row WHERE row.FIPS IS NOT NULL MERGE (c:Country {FIPS: row.FIPS, name: row.Name});",   
    )
    summary = result.consume()
    return summary

: 

In [ ]:
with driver.session(database="guillaume") as session:
    summary = session.execute_write(import_country)  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))